In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>"
)

In [ ]:
# Install Google Colab dependencies
# Note: this can take 30+ minutes (many of the dependencies include C++ code, which needs to be compiled)

# First install `sf`, `ragg` and `textshaping` and their system dependencies:
system("apt-get -y update && apt-get install -y  libudunits2-dev libgdal-dev libgeos-dev libproj-dev libharfbuzz-dev libfribidi-dev")
install.packages("sf")
install.packages("textshaping")
install.packages("ragg")

# Install system dependencies of some other R packages that Voyager either imports or suggests:
system("apt-get install -y libfribidi-dev libcairo2-dev libmagick++-dev")

# Install Voyager from Bioconductor:
install.packages("BiocManager")
BiocManager::install(version = "3.17", ask = FALSE, update = FALSE, Ncpus = 2)
BiocManager::install("scater")
system.time(
  BiocManager::install("Voyager", dependencies = TRUE, Ncpus = 2, update = FALSE)
)

packageVersion("Voyager")

# Introduction

The `SpatialFeatureExperiment` (SFE) and `Voyager` packages were originally developed around a relatively small Visium dataset as a proof of concept, and were hence not originally optimized for very large datasets. However, larger smFISH datasets with hundreds of thousands, sometimes over a million cells have already been produced and will soon be produced routinely. Among studies using smFISH-based spatial transcriptomics technologies that reported the number of cells per dataset, the number of cells per dataset has increased in the past years [@Moses2022-xz].
![alt](https://pachterlab.github.io/LP_2021/05-current-techs_files/figure-html/smfish-lm-cell-1.png)

In anticipation of large datasets, this vignette was produced using limited GitHub Actions resources (MacOS), which are 14 GB of RAM with 3 CPU cores and 14 GB of disk space, comparable to laptops. We therefore expect that the analyses in this vignette should scale to reasonably sized datasets.  

The dataset we use in this vignette is a MERFISH mouse liver dataset downloaded from the [Vizgen website](https://console.cloud.google.com/storage/browser/vz-liver-showcase/Liver1Slice1). We will use it to discuss some issues with large datasets and some upcoming features in the next release of `Voyager`. The gene count matrix and cell metadata (including centroid coordinates) were downloaded as CSV files and read into R. While 7 z-planes were imaged, cell segmentation is only available in one z-plane. The cell polygons are in HDF5 files, with one HDF5 file per field of view (FOV), and there are over 1000 FOVs in this dataset. Converting these HDF5 files into an `sf` data frame is not trivial. See the [vignette on creating a `SpatialFeatureExperiment` (SFE) object](https://pachterlab.github.io/voyager/articles/create_sfe.html#technology-specific-notes) for code used to do the conversion, and the polygons are included in the SFE object. The cell metadata already has cell volume. If the polygons are not used in the analyses, and the polygons can't be seen on a static plot with hundreds of thousands of cells anyway, then doing the conversion is optional. While the transcript spot locations are available, we cannot yet work with such a large point dataset.

Here we load the packages used. 

In [ ]:
library(Voyager)
library(SFEData)
library(SingleCellExperiment)
library(SpatialExperiment)
library(scater) 
library(ggplot2)
library(patchwork)
library(stringr)
library(spdep)
library(BiocParallel)
library(BiocSingular)
library(gstat)
library(BiocNeighbors)
library(sf)
library(automap)
theme_set(theme_bw())

In [ ]:
(sfe <- VizgenLiverData())

There are 395,215 cells in this dataset. Plotting them all as polygons takes a while, but it isn't too bad.

In [ ]:
plotGeometry(sfe, "cellSeg")

However, we do not wish to save this plot as PDF. To avoid this problem, we can either use the `scattermore = TRUE` argument in `plotSpatialFeature()` and plot the centroids since the polygons are hard to see anyway. 

Cell density can be vaguely seen in the plot above. Here we count the number of cells in bins to better visualize cell density.

In [ ]:
plotCellBin2D(sfe, bins = 300, hex = TRUE)

Cell density is for the most part homogeneous but shows some structure with denser regions that seem to relate to the blood vessels.

# Quality control

In [ ]:
names(colData(sfe))

Plotting almost 400,000 polygons is kind of slow but doable.

In [ ]:
system.time(
    print(plotSpatialFeature(sfe, "nCounts", colGeometryName = "cellSeg"))
)

Here nCounts kind of looks like salt and pepper. Using the [`scattermore`](https://github.com/exaexa/scattermore) package can speed up plotting a large number of points. In this non-interactive plot, the cell polygons are too small to see anyway, so plotting cell centroid points should be fine. 

In [ ]:
system.time({
    print(plotSpatialFeature(sfe, "nCounts", colGeometryName = "centroids",
                             scattermore = TRUE))
})

When run on our server, plotting almost 400,000 polygons took around 23 seconds, while using `geom_scattermore()` (`scattermore = TRUE`) took about 2 seconds. Since `geom_scattermore()` rasterizes the plot, the plot will be pixelated when zoomed in. 

While interactive data visualization is useful for ESDA, there is a need for static figures in publications. As of Voyager 1.2.0 (Bioconductor 3.17), a bounding box can be specified to zoom into the data.

In [ ]:
bbox_use <- c(xmin = 3000, xmax = 3500, ymin = 2500, ymax = 3000)

In [ ]:
plotSpatialFeature(sfe, "nCounts", colGeometryName = "cellSeg", bbox = bbox_use)

Much of the time making this plot is spent subsetting the `sf` data frame with the bounding box. Here, spatial autocorrelation is evident in the upper right region with smaller cells, but less so in the rest of this patch. nCounts seems to be related to cell size; larger cells seem to have more total counts.

Interactive data visualization is currently beyond the scope of `Voyager` or this vignette. There are existing tools for interactive visualization of highly multiplexed imaging data, such as [`MERmaid`](https://github.com/JEFworks-Lab/MERmaid) [@Wang2020-sv] for MERFISH data, [`TissUUmaps`](https://github.com/TissUUmaps/TissUUmapsCore) [@Pielawski2022-wk], and [`Visinity`](https://github.com/labsyspharm/visinity) [@Warchol2022-zw].

Since there aren't too many genes, all genes and negative control probes can be displayed:

In [ ]:
rownames(sfe)

The number of real genes is 347.

In [ ]:
n_panel <- 347

Next, we plot the distribution of nCounts divided by the number of genes in the panel, so this distribution is more comparable across datasets with different numbers of genes.

In [ ]:
colData(sfe)$nCounts_normed <- sfe$nCounts / n_panel
colData(sfe)$nGenes_normed <- sfe$nGenes / n_panel

In [ ]:
plotColDataHistogram(sfe, c("nCounts_normed", "nGenes_normed"))

As with the [Xenium dataset](https://pachterlab.github.io/voyager/articles/vig5_xenium.html#cells), there are mysterious regular notches in the histogram of the number of genes detected.

We also plot the number of genes detected per cell, with `geom_scattermore()`.

In [ ]:
plotSpatialFeature(sfe, "nGenes", colGeometryName = "centroids", scattermore = TRUE)

Similarly to nCounts, the points look intermingled. 

Distribution of cell volume in space:

In [ ]:
plotSpatialFeature(sfe, "volume", colGeometryName = "centroids", scattermore = TRUE)

Next, we explore how nCounts relates to nGenes:

In [ ]:
plotColDataBin2D(sfe, "nCounts", "nGenes")

There are two branches in this plot. 

How does cell size relate to nCounts?

In [ ]:
plotColDataBin2D(sfe, "volume", "nCounts")

The lower branch has the larger cells that don't tend to have more total counts, and the upper branch has larger cells that tend to have more total counts.

We also examine how cell size relates to number of genes detected:

In [ ]:
plotColDataBin2D(sfe, "volume", "nGenes")

There seem to be clusters that are possibly related to cell type. 

### Negative controls
Blank probes are used as negative controls.

In [ ]:
is_blank <- str_detect(rownames(sfe), "^Blank-")

In [ ]:
sfe <- addPerCellQCMetrics(sfe, subset = list(blank = is_blank))

In [ ]:
names(colData(sfe))

Total transcript counts from the blank probes:

In [ ]:
plotSpatialFeature(sfe, "subsets_blank_sum", colGeometryName = "centroids",
                   scattermore = TRUE)

Number of blank features detected per cell:

In [ ]:
plotSpatialFeature(sfe, "subsets_blank_detected", colGeometryName = "centroids",
                   scattermore = TRUE)

Percentage of blank features per cell:

In [ ]:
plotSpatialFeature(sfe, "subsets_blank_percent", colGeometryName = "centroids",
                   scattermore = TRUE)

The percentage is more interesting: within the tissue, cells with high percentage of blank counts are scattered like salt and pepper, but more of these cells are on the left edge of the tissue, the edges of FOVs, where the tissue itself doesn't end. 

Also plot histograms: 

In [ ]:
plotColDataHistogram(sfe, paste0("subsets_blank_", c("sum", "detected", "percent")))

The NA's are cells without any transcript detected. 

In [ ]:
mean(sfe$subsets_blank_sum > 0)

Unlike in the Xenium dataset, here most cells have at least one blank count. By log transforming, the zeroes are removed from the plot. 

In [ ]:
plotColDataHistogram(sfe, "subsets_blank_percent") +
    scale_x_log10() +
    annotation_logticks()

A small percentage of blank counts is acceptable. So we will remove the outlier based on the distribution of the percentage when it's greater than zero. How does the blank percentage relate to total counts?

In [ ]:
plotColDataBin2D(sfe, "nCounts", "subsets_blank_percent") +
    scale_fill_viridis_c()

The outliers in percentage of blank counts have low total counts. But there are some seemingly real cells with sizable nCounts which have a relatively high percentage of blank counts. Since the distribution of this percentage has a very long tail, we log transform it when finding outliers.

In [ ]:
get_neg_ctrl_outliers <- function(col, sfe, nmads = 3, log = FALSE) {
    inds <- colData(sfe)$nCounts > 0 & colData(sfe)[[col]] > 0
    df <- colData(sfe)[inds,]
    outlier_inds <- isOutlier(df[[col]], type = "higher", nmads = nmads, log = log)
    outliers <- rownames(df)[outlier_inds]
    col2 <- str_remove(col, "^subsets_")
    col2 <- str_remove(col2, "_percent$")
    new_colname <- paste("is", col2, "outlier", sep = "_")
    colData(sfe)[[new_colname]] <- colnames(sfe) %in% outliers
    sfe
}

In [ ]:
sfe <- get_neg_ctrl_outliers("subsets_blank_percent", sfe, log = TRUE)

What proportion of all cells are outliers?

In [ ]:
mean(sfe$is_blank_outlier)

What's the cutoff for outlier?

In [ ]:
min(sfe$subsets_blank_percent[sfe$is_blank_outlier])

Remove the outliers and empty cells:

In [ ]:
(sfe <- sfe[, !sfe$is_blank_outlier & sfe$nCounts > 0])

There still are over 390,000 cells left after removing the outliers.

## Genes
Here we look at the mean and variance of each gene:

In [ ]:
rowData(sfe)$means <- rowMeans(counts(sfe))
rowData(sfe)$vars <- rowVars(counts(sfe))

In [ ]:
rowData(sfe)$is_blank <- is_blank
plotRowData(sfe, x = "means", y = "is_blank") +
    scale_y_log10() +
    annotation_logticks(sides = "b")

Most genes display higher mean expression than blanks, but there is considerable overlap in the distribution, probably because some genes expressed at lower levels or in fewer cells are included.

Here the "real" genes and negative controls are plotted in different colors:

In [ ]:
plotRowData(sfe, x = "means", y = "vars", colour_by = "is_blank") +
    geom_abline(slope = 1, intercept = 0, color = "red") +
    scale_x_log10() + scale_y_log10() +
    annotation_logticks() +
    coord_equal()

The red line $y = x$ is expected when the data follows a Poisson distribution. 

In [ ]:
as.data.frame(rowData(sfe)[is_blank,]) |> 
    ggplot(aes(means, vars)) +
    geom_point() +
    geom_abline(slope = 1, intercept = 0, color = "red") +
    scale_x_log10() + scale_y_log10() +
    annotation_logticks() +
    coord_equal()

When zoomed in, the blanks are somewhat overdispersed.

# Spatial autocorrelation of QC metrics

Again, we plot that zoomed in patch to visually inspect cell-cell contiguity:

In [ ]:
plotGeometry(sfe, "cellSeg", bbox = bbox_use)

There are quite a few cells that are not contiguous to any other cell, and cell segmentation is imperfect, so purely using `poly2nb()` is problematic. In the next release, we might implement a way to blend a polygon contiguity graph with some other graph in case of singletons. For now, we use k nearest neighbors with $k = 5$, which seems like a reasonable approximation of contiguity based on the visual inspection. 

As of Voyager 1.2.0 (Bioconductor 3.17), `findSpatialNeighbors()` by default uses `BiocNeighbors` for k nearest neighbors and distance neighbors and saving the distances between neighbors. This bypasses the most time consuming step in `spdep` when calculating distance based edge weights, which is to compute distance, hence greatly speeding up computation.

In [ ]:
system.time(
    colGraph(sfe, "knn5") <- findSpatialNeighbors(sfe, method = "knearneigh", 
                                                  dist_type = "idw", k = 5, 
                                                  style = "W")
)

With the spatial neighborhood graph, we can compute Moran's I for QC metrics.

In [ ]:
sfe <- colDataMoransI(sfe, c("nCounts", "nGenes", "volume"), 
                      colGraphName = "knn5")

In [ ]:
colFeatureData(sfe)[c("nCounts", "nGenes", "volume"),]

Unlike the other smFISH-based datasets on this website, nCounts and nGenes have sizable negative Moran's I's, which is closer to 0 for volume. It would be interesting to compare these metrics across different tissues, as we add more datasets to `SFEData` in future releases.

Also check local Moran's I, since in that little patch we examined above, some regions may have more positive spatial autocorrelation.

In [ ]:
sfe <- colDataUnivariate(sfe, type = "localmoran", 
                         features = c("nCounts", "nGenes", "volume"),
                         colGraphName = "knn5")

In [ ]:
plotLocalResult(sfe, "localmoran", c("nCounts", "nGenes", "volume"),
                colGeometryName = "centroids", scattermore = TRUE,
                ncol = 2, divergent = TRUE, diverge_center = 0)

There are some niches around smaller blood vessels with positive local Moran's I for nCounts and nGenes. This is most likely due to the more homogeneous endothelial cells compared to hepatocytes.

# Moran's I

In [ ]:
sfe <- logNormCounts(sfe)

In [ ]:
system.time(
    sfe <- runMoransI(sfe, BPPARAM = MulticoreParam(2))
)

It's actually not as slow as I thought for almost 400,000 cells. How are Moran's I's distributed for real genes and blank probes?

In [ ]:
plotRowData(sfe, x = "moran_sample01", y = "is_blank") +
    geom_hline(yintercept = 0, linetype = 2)

The blanks are clustered tightly around 0. The vast majority of real genes have positive spatial autocorrelation, some quite strong. But some genes have negative spatial autocorrelation, although it may or may not be statistically significant.

Plot the top genes with positive spatial autocorrelation:

In [ ]:
top_moran <- rownames(sfe)[order(rowData(sfe)$moran_sample01, decreasing = TRUE)[1:6]]
plotSpatialFeature(sfe, top_moran, colGeometryName = "centroids", scattermore = TRUE,
                   ncol = 2)

Unlike in the other smFISH-based cancer datasets on this dataset, the genes with the highest Moran's I highlight different histological regions. Some probably for zones in the hepatic lobule, and some for blood vessels. It would be interesting to compare spatial autocorrelation of marker genes among different tissues and cell types.

Negative Moran's I means that nearby cells tend to be more dissimilar to each other. That would be hard to see when plotting the whole tissue section, so we will use that bounding box again. The gene with the most negative Moran's I is compared to one with Moran's I closest to 0.

In [ ]:
bottom_moran <- rownames(sfe)[order(rowData(sfe)$moran_sample01)[1]]
bottom_abs_moran <- rownames(sfe)[order(abs(rowData(sfe)$moran_sample01))[1]]
plotSpatialFeature(sfe, c(bottom_moran, bottom_abs_moran), 
                   colGeometryName = "cellSeg", bbox = bbox_use)

As expected, the feature with Moran's I closest to 0 is a blank.

# Spatial autocorrelation at larger length scales
The k nearest neighbor graph used above only concerns 5 cells around each cell, which is a very small neighborhood, over a very small length scale. In the current release of `Voyager`, a correlogram can be computed to get a sense of the length scale of spatial autocorrelation. However, since finding the lag values over higher and higher orders of neighborhoods is very slow for such a large number of cells for higher orders, the correlogram is not very helpful here. In this section, we use some other methods involving binning to explore spatial autocorrelation at larger length scales.

## Binning
The `sf` package can create polygons in a grid, with which we can bin the cells and their attributes and gene expressions. Here we make a 100 by 100 hexagonal grid in the bounding box of the cell centroids.

In [ ]:
(bins <- st_make_grid(colGeometry(sfe, "centroids"), n = 100, square = FALSE))

Here we use this grid to bin the QC metrics by averaging the values from the cells. Since bins not completely covered by tissue have fewer cells, the mean may be less susceptible to edge effect than the sum, as bins near the edge will have lower sums, which may spuriously increase Moran's I. 

In [ ]:
df <- cbind(colGeometry(sfe, "centroids"), colData(sfe)[,c("nCounts", "nGenes", "volume")])
df_binned <- aggregate(df, bins, FUN = mean)
# Remove bins not containing cells
df_binned <- df_binned[!is.na(df_binned$nCounts),]

Plot the binned values:

In [ ]:
# Not using facet_wrap to give each panel its own color scale
plts <- lapply(c("nCounts", "nGenes", "volume"), function(f) {
    ggplot(df_binned[,f]) + geom_sf(aes(fill = .data[[f]]), linewidth = 0) +
        scale_fill_distiller(palette = "Blues", direction = 1) + theme_void()
})
wrap_plots(plts, nrow = 2)

There's an outlier bin not as evident when plotting single cells. There's still some edge effect around blood vessels. This might be truly edge effect, or that endothelial cells tend to have lower values in all 3 variables here.

Then compute Moran's I over the binned data, with contiguity neighborhoods. `zero.policy = TRUE` because there are some bins with no neighbors.

In [ ]:
nb <- poly2nb(df_binned)
listw <- nb2listw(nb, zero.policy = TRUE)

In [ ]:
calculateMoransI(t(as.matrix(st_drop_geometry(df_binned[,c("nCounts", "nGenes", "volume")]))),
                 listw = listw, zero.policy = TRUE)

At a larger length scale, Moran's I becomes positive. Comparing Moran's I across different sized bins can give a sense of the length scale of spatial autocorrelation. However, there are problems with binning to watch out for:

1. Edge effect, especially when using the sum when binning
2. Which function to use to aggregate the values when binning
3. When using a rectangular grid, whether to use rook or queen neighbors. Rook means two cells are neighbors if they share an edge, while queen means they are neighbors even if they merely share a vertex. 

While binning can greatly speed up computation of spatial autocorrelation metrics for larger datasets, and is in fact used for Moran's I in Seurat, it can be used for smaller datasets to find length scales of spatial autocorrelation. On the other hand, as seen here, Moran's I can flip signs at different length scales, so for larger datasets, exploring spatial autocorrelation at the cell level would still be interesting.

## Semivariogram
In geostatistical data, an underlying spatial process is sampled at known locations. Kriging uses a Gaussian process to interpolate the values between the sample locations, and the semivariogram is used to model the spatial dependency between the locations as the covariance of the Gaussian process. When not kriging, the semivariogram can be used as an exploratory data analysis tool to find the length scale and anisotropy of spatial autocorrelation. One of the classic R packages in the geostatistical tradition is [`gstat`](https://cran.r-project.org/web/packages/gstat/index.html), which we use here to find semivariograms, defined as

$$
\gamma(t) = \frac 1 2 \mathrm{Var}(X_t - X_0),
$$

where $X$ is the value such as gene expression, and $t$ is a spatial vector. $X_0$ is the value at a location of interest, and $X_t$ is the value lagged by $t$. With positive spatial autocorrelation, the variance would be smaller among nearby values, so the variogram would increase with distance, eventually leveling off when the distance is beyond the length scale of spatial autocorrelation. The "semi" comes from the 1/2.

In the next release of Voyager, this will be part of `runUnivariate()` so the learning curve for users will be minimal.

First we find an empirical variogram assuming that it's the same in all directions. Here the data is binned at distance intervals, so this is much faster than the correlogram at the cell level. The `width` argument controlls the bin size. The `cutoff` argument is the maximum distance to consider. Here we use the defaults. The first argument is a formula; covariates can be specified, but is not done here.

With different widths and cutoffs, the variogram can be estimated at different length scales. The `gstat` package can also fit a model to the empirical variogram. See `vgm()` for the different types of models. The [`automap`](https://cran.r-project.org/web/packages/automap/) package can choose a model for the user, and is used here. 

Unfortunately, `gstat` doesn't scale to 400,000 cells, although it worked for 100,000 cells in the other smFISH-based datasets on this website. But since the variogram is used to explore larger length scales here anyway, we use the binned data here, but the problems with binning will apply.

In [ ]:
# as_Spatial since automap uses old fashioned sp, the predecessor of sf
v <- autofitVariogram(nCounts ~ 1, as_Spatial(df_binned))
plot(v)

The numbers in this plot are the number of pairs in each distance bin. The variogram is not 0 at 0 distance; this is the variance within the bin size, called the nugget. The variogram levels off at greater distance, and the value of the variogram where is levels off is the sill. Range is where the variogram is leveling off, indicating length scale of spatial autocorrelation; the range from visual inspection appears closer to 1000 while the model somehow indicates 423. 

A variogram map can be made to see how spatial autocorrelation may differ in different directions, i.e. anisotropy

In [ ]:
v2 <- variogram(nCounts ~ 1, data = df_binned, width = 300, cutoff = 4500, map = TRUE)
plot(v2)

Here apparently there's no anisotropy at shorter length scales, but there may be some artifact from the hexagonal bins. Going beyond 2000 (whatever unit), the variance drops at the northwest and southeast direction but not in other directions, perhaps related to the repetitiveness of the hepatic lobules and the general NE/SW direction of the blood vessels seen in the previous plots. 

The variogram can also be calculated at specified angles, here selected as sides of the hexagon:

In [ ]:
v3 <- variogram(nCounts ~ 1, df_binned, alpha = c(30, 90, 150))
v3_model <- fit.variogram(v3, vgm("Ste"))
plot(v3, v3_model)

The variogram rises when going beyond 2000 at 30 and 90 degrees and drops at 150 degrees. This is consistent with the variogram map. These differences are averaged out in the omni-directional variogram. `gstat` does not fit anisotropy parameters, so the fitted curve is omni-directional. It fits pretty well below 2000. This is only for nCounts, and may differ for other QC metrics and genes. What if anisotropy varies in space? A problem with the variogram is that it's global, giving one result for the entire dataset, albeit more nuanced than just a number as in Moran's I, because kriging assumes that the data is intrinsically stationary, meaning that the same variogram model applies everywhere, and that spatial dependence only depends on lag between two observations.

Voyager 1.2.0 implements `ggplot2` based plotting functions to make better looking and more customizable plots of the variograms for SFE objects. However, the binned data is not in an SFE object. We are considering writing a method to spatially bin all cell level data in an SFE object. Here `gstat` is using `lattice`, a predecessor of `ggplot2` to make facetted plots and has been superseded by `ggplot2`. `gstat` is one of the oldest R packages still on CRAN, dating back to the days of S (prequel of R), although its oldest archive on CRAN is from 2003. `spdep` is also really old; its oldest archive on CRAN is from 2002, but it's still in active development. In using these time honored packages and methods (Moran's I and Geary's C themselves date back to the 1950s and their modern form date back to 1969 [@Cliff1969-wp; @Bivand2013-jx]) on the cool new spatial transcriptomics dataset, we are participating in a glorious tradition, which we will further develop as a spatial analysis tradition forms around spatial -omics data analysis.

# PCA for larger datasets
There are many ways to do PCA in R, and the [`BiocSingular`](https://bioconductor.org/packages/release/bioc/html/BiocSingular.html) package makes a number of different methods available with a consistent user interface, and it supports out of memory data in `DelayedArray`. According to [this benchmark](https://slowkow.com/notes/pca-benchmark/), `stats::prcomp()` shipped with R is rather slow for larger datasets. The fastest methods are `irlba::irlba()` and `RSpectra::svds()`, and the former is supported by `BiocSingular`. So here we use IRLBA and see how long it takes. Many PCA algorithms involve repeated matrix multiplications. R does not come with optimized BLAS and LAPACK, for portability reasons. However, the BLAS and LAPACK used by R can be changed to an optimized one (here's [how to do it](https://cran.r-project.org/doc/manuals/r-release/R-admin.html#Shared-BLAS)), which will speed up matrix multiplication. 

In [ ]:
set.seed(29)
system.time(
    sfe <- runPCA(sfe, ncomponents = 20, subset_row = !is_blank,
                  exprs_values = "logcounts",
                  scale = TRUE, BSPARAM = IrlbaParam())
)
gc()

That's pretty quick for almost 400,000 cells, but there aren't that many genes here. Use the elbow plot to see variance explained by each PC:

In [ ]:
ElbowPlot(sfe)

Plot top gene loadings in each PC

In [ ]:
plotDimLoadings(sfe)

Many of these genes seem to be related to the endothelium.

Plot the first 4 PCs in space

In [ ]:
spatialReducedDim(sfe, "PCA", 4, colGeometryName = "centroids", scattermore = TRUE,
                  divergent = TRUE, diverge_center = 0)

PC1 and PC4 highlight the major blood vessels, while PC2 and PC3 have less spatial structure. While in the CosMX and Xenium datasets on this website, the top PCs have clear spatial structures despite the absence of spatial information in non-spatial PCA because of clear spatial compartments for some cell types, which does not seem to be the case in this dataset except for the blood vessels. We have seen above that some genes have strong spatial structures. There are some methods for spatially informed PCA, such as MULTISPATI PCA [@Dray2008-en] in the [`adespatial`](https://cran.r-project.org/web/packages/adespatial/index.html) package, which seeks to maximize both variance (as in non-spatial PCA) and Moran's I on each PC. Unlike traditional PCs, where all eigenvalues, signifying variance explained, are positive, MULTISPATI PCA can have negative eigenvalues, which signify negative spatial autocorrelation. The PCs from MULTISPATI PCA with positive eigenvalues are also more spatially coherent than those from non-spatial PCA. For the CosMX and Xenium datasets on this website, the spatial coherence from MULTISPATI might not make a difference, but it might make a difference in this dataset where non-spatial PCs don't show as much spatial structure, at least at the larger scale over the entire tissue section. Voyager 1.2.0 (Bioconductor 3.17) has a faster implementation of MULTISPATI PCA than that of `adespatial`, and is demonstrated in this dataset in [another vignette](https://pachterlab.github.io/voyager/articles/multispati.html).

While PC2 and PC3 don't seem to have large scale spatial structure, they may have more local spatial structure not obvious from plotting the entire section, so we zoom into the bounding box:

In [ ]:
spatialReducedDim(sfe, "PCA", ncomponents = 2:3, colGeometryName = "cellSeg",
                  bbox = bbox_use, divergent = TRUE, diverge_center = 0)

There's some spatial structure in PC2 and PC3 at a smaller scale, perhaps some negative spatial autocorrelation.

Like global Moran's I, PCA and MULTISPATI PCA return one result for the entire dataset. In contrast, [geographically weighted PCA (GWPCA)](https://rdrr.io/cran/GWmodel/man/gwpca.html) [@Harris2015-sr] can account for spatial heterogeneity. GWPCA runs PCA at each spatial location using only the nearby locations weighed by a kernel. The different locations can have different PCs, and the results can be visualized with "winning variables" in each PC, i.e. plotting which feature has the highest loading in each PC in space. This most likely doesn't scale to 400,000 cells, but it would still be interesting when performed on spatially binned data. GWPCA might be added in Bioconductor 3.18 and would require some changes in the user interface, because GWPCA is about the features rather than cell embeddings.

# More challenges from large datasets
Here, despite the numerous cells, the data was loaded into memory. What if the data doesn't fit into memory? We might write a new vignette with `DelayedArray` demonstrating out of memory data analysis for Bioconductor 3.18. This is already supported by `SingleCellExperiment`, which SFE inherits from. However, the geometries, graphs, and local results can take up a lot memory as well. These can possibly be stored in SQL databases and operated on with [`SQLDataFrame`](https://bioconductor.org/packages/release/bioc/html/SQLDataFrame.html). The geometric operations can be handled by [`sedona`](https://github.com/apache/incubator-sedona), although the options are limited compared to GEOS, which performs geometric operations behind the scene for `sf`.

Another question can be raised about large spatial transcriptomics data: Is it still a good idea to analyze the entire dataset at once? There must be many interesting and unique neighborhoods that might not get the attention they deserve when the whole dataset is analyzed at once. After all, in the geographical space, national level data is usually not analyzed at the block resolution, although a reason for this is privacy of the subjects. County resolution is often used, but there aren't hundreds of thousands of counties. Many analyses are done for cities and counties with neighborhood resolution; using the largest geographical unit isn't always the most relevant. Back to histological space: How to aggregate cells into larger spatial units? How to decide which scale of spatial units (analogous to nation vs state vs county and etc) is relevant? We have traditional anatomical ontologies, such as from the Allen Brain Atlas, but this isn't available for all tissues. Also, with more single cell -omics data, the traditional ontologies can be improved.

Furthermore, there are 3D thick section single cell resolution spatial transcriptomics data, such as from STARmap [@Wang2018-cv] and EASI-FISH [@Wang2021-fv], although the vast majority of spatial -omics data are from thin sections pretty much _de facto_ 2D. As we mostly live on the surface of Earth, there are many more 2D geospatial resources than 3D. However, some methods can in principle be applied to 3D and the existing software primarily made for 2D data might work. For example, GEOS supports 3D data, so in principle 3D geometries in `sf` should work, although there's little documentation on this. Also, k nearest neighbor, Moran's I, variograms, and etc. should in principle work in 3D, and `gstat` officially supports 3D. These are more challenges related to 3D data:

1. Even when multiple z-planes are imaged, the resolution is much lower in the z direction than the x and y directions. Then should the z-plane be treated as an attribute or as a coordinate?
2. How to make static plots for 3D data for publications? This is more complicated than plotting z-planes separately, since a 3D block can be sectioned from any other direction. Also for interactive visualization, we need to somehow see through the tissue. 

Finally, the geospatial tradition is only one tradition relevant to large spatial data, and at present `Voyager` only works with vector data. We are uncertain whether raster should be added in a later version, and there are existing tools for large raster data as well, such as [TileDB](https://docs.tiledb.com/geospatial/). Other traditions can be relevant, such as astronomy and image processing, but those are beyond the scope of this package.

# Session Info

In [ ]:
sessionInfo()

# References